### Trabalho 3 - Inversores
###### Grupo 19

Tiago Passos Rodrigues - A96414

### Enunciado


O seguinte sistema dinâmico denota 4 inversores ($\,A, B, C, D\,$) que lêm um bit num canal input e escrevem num canal output uma transformação desse bit.

   i. Cada inversor tem um bit $s$ de estado, inicializado  com um valor aleatório.
   
   ii. Cada inversor é regido pelas seguintes transformações
$$\mathbf{invert}\mathtt(in,out)$$
$$x \gets \mathsf{read}(\mathtt{in})$$
$$s \gets \neg x\;\;\|\;\; s\gets s\oplus x$$            
$$\mathsf{write}(\mathtt{out},s)$$
   iii. A escolha neste comando é sempre determinística; isto é, em cada inversor a escolha do comando a executar é sempre a mesma. Porém qual é essa escolha é determinada aleatoriamente na inicializarão do sistema.
   
   iii. O estado do sistema é um duplo definido pelos 4 bits $s$, e é inicializado com um vetor aleatório em $\{0,1\}^4\;$.
   
   iv. O sistema termina em ERRO quando o estado do sistema for $\,(0,0,0,0)\,$.
                    
   1. Construa um SFOTS que descreva este sistema e implemente este sistema, numa abordagem BMC (“bouded model checker”) num traço com $\,n\,$ estados.
   2. Verifique se o sistema é seguro usando BMC, k-indução ou model checking com interpolantes.

### Implementação

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools
n = 3


def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),BVType(n))
    return state

'''
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i),BVType(n))
    state['Inv_A'] = Symbol('A'+str(i),BVType(n))
    state['Inv_B'] = Symbol('B'+str(i),BVType(n))
    state['Inv_C'] = Symbol('C'+str(i),BVType(n))
    state['Inv_D'] = Symbol('D'+str(i),BVType(n))
    return state
'''

"\ndef declare(i):\n    state = {}\n    state['pc'] = Symbol('pc'+str(i),BVType(n))\n    state['Inv_A'] = Symbol('A'+str(i),BVType(n))\n    state['Inv_B'] = Symbol('B'+str(i),BVType(n))\n    state['Inv_C'] = Symbol('C'+str(i),BVType(n))\n    state['Inv_D'] = Symbol('D'+str(i),BVType(n))\n    return state\n"

### Criação do SFOTS

In [2]:
'''
t0 = And(Equals(curr['pc'],Int(0)), Equals(prox['pc'],Int(1)), Ite(TRUE(),
                                                                Equals(prox['Inv_B'], BVXor(curr['Inv_A'],curr['Inv_B'])),
                                                                Ite(Equals(curr['Inv_A'],BVOne(1)),
                                                                                Equals(prox['Inv_B'],BVZero(1)),
                                                                                Equals(prox['Inv_B'],BVOne(1)))))
'''
import random
# valor aleatorio entre 0 e 1 para determinar qual a funcao a usar |¬x ∥ s←s⊕x |
funcao = random.randint(0, 1)

def trans1(curr,prox):

    t0 = And(Equals(curr['pc'],BV(0,n)), Equals(prox['pc'],BV(1,3)), Equals(Int(funcao),Int(0)),
                                                                Ite(Equals(curr['Inv_A'],BVOne(n)),
                                                                                Equals(prox['Inv_B'],BVZero(n)),
                                                                                Equals(prox['Inv_B'],BVOne(n))),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_C'],curr['Inv_C']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t1 = And(Equals(curr['pc'],BV(0,n)),Equals(prox['pc'],BV(1,n)), Equals(Int(funcao),Int(1)), 
             Equals(prox['Inv_B'],BVXor(curr['Inv_A'],curr['Inv_B'])),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_C'],curr['Inv_C']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t2 = And(Equals(curr['pc'],BV(1,n)),Equals(prox['pc'],BV(2,n)), Equals(Int(funcao),Int(0)), Ite(Equals(curr['Inv_B'],BVOne(n)),
                                                                                Equals(prox['Inv_C'],BVZero(n)),
                                                                                Equals(prox['Inv_C'],BVOne(n))),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t3 = And(Equals(curr['pc'],BV(1,n)), Equals(Int(funcao),Int(1)), Equals(prox['pc'],BV(2,n)), Equals(prox['Inv_C'],BVXor(curr['Inv_B'],curr['Inv_C'])),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_D'],curr['Inv_D']))
    
    t4 = And(Equals(curr['pc'],BV(2,n)), Equals(Int(funcao),Int(0)), Equals(prox['pc'],BV(3,n)), Ite(Equals(curr['Inv_C'],BVOne(n)),
                                                                                Equals(prox['Inv_D'],BVZero(n)),
                                                                                Equals(prox['Inv_D'],BVOne(n))),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))
    
    t5 = And(Equals(curr['pc'],BV(2,n)), Equals(Int(funcao),Int(1)), Equals(prox['pc'],BV(3,n)), Equals(prox['Inv_D'],BVXor(curr['Inv_C'],curr['Inv_D'])),
            Equals(prox['Inv_A'],curr['Inv_A']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))
    
    t6 = And(Equals(curr['pc'],BV(3,n)), Equals(Int(funcao),Int(0)), Equals(prox['pc'],BV(0,n)), Ite(Equals(curr['Inv_D'],BVOne(n)),
                                                                                Equals(prox['Inv_A'],BVZero(n)),
                                                                                Equals(prox['Inv_A'],BVOne(n))),
            Equals(prox['Inv_D'],curr['Inv_D']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))
    
    t7 = And(Equals(curr['pc'],BV(3,n)), Equals(Int(funcao),Int(1)), Equals(prox['pc'],BV(0,n)), Equals(prox['Inv_A'],BVXor(curr['Inv_D'],curr['Inv_A'])),
             Equals(prox['Inv_D'],curr['Inv_D']),Equals(prox['Inv_B'],curr['Inv_B']),Equals(prox['Inv_C'],curr['Inv_C']))

    return Or(t0,t1,t2,t3,t4,t5,t6,t7)
        
def init1(state):
    return And(Equals(state['pc'],BV(0,n)), 
          Equals(state['Inv_A'],BV(random.randint(0, 1),n)),
          Equals(state['Inv_B'],BV(random.randint(0, 1),n)),
          Equals(state['Inv_C'],BV(random.randint(0, 1),n)),
          Equals(state['Inv_D'],BV(random.randint(0, 1),n)),
           Not(And(Equals(state['Inv_A'],BVZero(n)),Equals(state['Inv_B'],BVZero(n)),Equals(state['Inv_C'],BVZero(n)),Equals(state['Inv_D'],BVZero(n))))
            )

def error1(state):
    return And(Equals(state['Inv_A'],BVZero(n)),
               Equals(state['Inv_B'],BVZero(n)),
               Equals(state['Inv_C'],BVZero(n)),
               Equals(state['Inv_D'],BVZero(n))
              )


In [3]:
def genTrace(vars,init,trans,error,k):
    with Solver(name="z3") as s:
        
        X = [genState(vars,'X',i) for i in range(k+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(k) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(k):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

print(funcao)
genTrace(['Inv_A','Inv_B','Inv_C','Inv_D','pc'],init1,trans1,error1,50)

1
Estado: 0
           Inv_A = 1_3
           Inv_B = 1_3
           Inv_C = 1_3
           Inv_D = 0_3
           pc = 0_3
Estado: 1
           Inv_A = 1_3
           Inv_B = 0_3
           Inv_C = 1_3
           Inv_D = 0_3
           pc = 1_3
Estado: 2
           Inv_A = 1_3
           Inv_B = 0_3
           Inv_C = 1_3
           Inv_D = 0_3
           pc = 2_3
Estado: 3
           Inv_A = 1_3
           Inv_B = 0_3
           Inv_C = 1_3
           Inv_D = 1_3
           pc = 3_3
Estado: 4
           Inv_A = 0_3
           Inv_B = 0_3
           Inv_C = 1_3
           Inv_D = 1_3
           pc = 0_3
Estado: 5
           Inv_A = 0_3
           Inv_B = 0_3
           Inv_C = 1_3
           Inv_D = 1_3
           pc = 1_3
Estado: 6
           Inv_A = 0_3
           Inv_B = 0_3
           Inv_C = 1_3
           Inv_D = 1_3
           pc = 2_3
Estado: 7
           Inv_A = 0_3
           Inv_B = 0_3
           Inv_C = 1_3
           Inv_D = 0_3
           pc = 3_3
Estado: 8
           I

### K-Indução

In [4]:
def kinduction_always(vars,init,trans,inv,k):

    with Solver(name="z3") as s:

        # create k+1 copies of the state vars
        states = [genState(vars,'X',i) for i in range(k)]
        
        # check for first k states
        s.push()
        s.add_assertion(init(states[0]))
        for i in range(k-1):
            s.add_assertion(trans(states[i], states[i+1]))
        s.add_assertion(Or([Not(inv(states[i])) for i in range(k)]))
        if s.solve():
            print("a prop n é valida nos primeiros", k ,"estados com a funcao", funcao)
            for v in states[0]:
                print(v, "=", s.get_value(states[0][v]))
            return
        s.pop()
        
        # check inductive step
        s.push()
        for t in range(k):
        #passo induçao
            s.add_assertion(inv(states[t]))
            s.add_assertion(trans(states[t],states[t+1]))
        s.add_assertion(Not(inv(states[k])))
        
        if s.solve():
            print("prop n é valida")
            for k in states[t]:
                print(k, "=",s.get_value(states[t][k]))
            return
        print("prop valida")
    
kinduction_always(['Inv_A','Inv_B','Inv_C','Inv_D','pc'],init1,trans1,error1,99)

a prop n é valida nos primeiros 99 estados com a funcao 1
Inv_A = 1_3
Inv_B = 1_3
Inv_C = 0_3
Inv_D = 1_3
pc = 0_3


###  Model checking com interpolantes

In [5]:
def invert(trans):
    return (lambda c, p: trans(p,c))

def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

In [6]:
def model_checking(vars,init,trans,error,N,M):
    with Solver(name="z3") as s:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        #print(order)
        for (n,m) in order:
            
            Tn = And([trans(X[i],X[i+1]) for i in range(n)])
            I = init(X[0])
            Rn = And(I,Tn)
            
            Bm = And([invert(trans)(Y[i],Y[i+1]) for i in range(m)])
            E = error(Y[0])
            Um = And(E,Bm)
            
            Vnm = And(Rn,same(X[n],Y[m]),Um)
            
            if s.solve([Vnm]):
                print("unsafe")
                return
            else:                                                  #Vnm insat
                C = binary_interpolant(And(Rn,same(X[n],Y[m])),Um)
                if C is None:
                    print("interpolante none")
                    break
                C0 = rename(C,X[0])
                C1 = rename(C,X[1])
                T = trans(X[0],X[1])
                
                if not s.solve([C0,T,Not(C1)]):                    # C é invariante de T
                    print("safe")
                    return
                else:
                    S = rename(C,X[n])
                    while True:
                        A = And(S,trans(X[n],Y[m]))
                        if s.solve([A,Um]):
                            print("nao é possivel majorar")
                            break
                        else:
                            Cnew = binary_interpolant(A,Um)
                            Cn = rename(Cnew,X[n])
                            if s.solve([Cn,Not(S)]):      # se Cn -> S não é Taut
                                S = Or(S,Cn)
                            else:                         # S foi encontrado
                                print("safe")
                                return
#######

model_checking(['Inv_A','Inv_B','Inv_C','Inv_D','pc'], init1, trans1, error1, 50, 50)

NoSolverAvailableError: No Interpolator is available for logic QF_AUFBVLIRA